# 🎵 Emotion Classifier using DEAM Dataset (Valence/Arousal Regression)
This notebook loads the DEAM dataset, processes audio into mel-spectrograms, trains a CNN to predict valence/arousal, and visualizes results.

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Set paths
AUDIO_DIR = "MEMD_audio"
ANNOTATIONS_CSV = "annotations\annotations averaged per song\song_level\static_annotations_averaged_songs_1_2000.csv"

# Load annotations
annotations = pd.read_csv(ANNOTATIONS_CSV)[['song_id', 'valence_mean', 'arousal_mean']]
annotations.head()

In [ ]:
# Convert audio to mel-spectrogram
def audio_to_mel_spectrogram(file_path, sr=22050, duration=30, n_mels=128):
    y, _ = librosa.load(file_path, sr=sr, duration=duration)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    S_dB = librosa.power_to_db(S, ref=np.max)
    return S_dB

In [ ]:
X = []
y = []
examples = []

print("Processing audio files...")

for idx, row in annotations.iterrows():
    file_id = str(row['song_id']).zfill(4)
    file_path = os.path.join(AUDIO_DIR, f"{file_id}.mp3")
    if os.path.exists(file_path):
        try:
            mel = audio_to_mel_spectrogram(file_path)
            mel = librosa.util.fix_length(mel, 128, axis=1)
            X.append(mel)
            y.append([row['valence_mean'], row['arousal_mean']])
            if len(examples) < 6:
                examples.append((mel, row['valence_mean'], row['arousal_mean'], file_id))
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

X = np.array(X)
y = np.array(y)

X = (X - X.min()) / (X.max() - X.min())
X = X[..., np.newaxis]


In [ ]:
# Visualize spectrograms
fig, axs = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle("Sample Mel-Spectrograms with Valence & Arousal", fontsize=16)

for i, (mel, val, aro, fid) in enumerate(examples):
    ax = axs[i//3, i%3]
    img = librosa.display.specshow(mel, sr=22050, x_axis='time', y_axis='mel', ax=ax)
    ax.set_title(f"ID: {fid} | Val: {val:.2f}, Aro: {aro:.2f}")
    ax.set_xlabel("")
    ax.set_ylabel("")

fig.colorbar(img, ax=axs, format="%+2.0f dB")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)

In [ ]:
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')
plt.grid(True)
plt.show()

In [ ]:
pred = model.predict(X_test[:5])
for i in range(5):
    print(f"Predicted: {pred[i]}, Actual: {y_test[i]}")